# Cross-Match KOA and Megafile

This code cross-matches the file that contains all the information for the HGCA stars, or the 'megafile', and the file which contains all the observations that have been reduced in KOA.

Because the megafile stars are by definition accelerating, it is important that the RA and Dec of these files is converted to be correct for the date of the Keck observations. Because it would take to long to convert every RA to every line in the megafile we only convert by each date in the megafile and then check the stars en masse. 

For the observation time in the megafile we use the average epoch of the Gaia RA and Dec. 

In [120]:
# Necessary Modules 
import pandas as pd
from astropy.coordinates import SkyCoord, Distance
from astropy import units as u
from astropy.time import Time
import numpy as np
import time
import glob

In [121]:
# Reading in the HGCA megafile
megafile = pd.read_csv('/Users/Jess/HGCA_survey_paper/megafile.csv')
print('There are ' + str(len(megafile)) + ' HCGA stars.')

There are 115663 HCGA stars.


In [122]:
# Reading in KOA file
koafile = pd.read_csv('/Users/Jess/Downloads/new_keck_stars_edited.csv')
print('There are ' + str(len(koafile)) + ' files in the KOA database.')

There are 335474 files in the KOA database.


In [123]:
# Create a list of unique observation dates
myset = set(koafile['date'])
myset = list(myset)
print('The KOA list of files has ' + str(len(koafile['date'])) + ' files, which have ' + str(len(myset)) + ' unique observation dates.')

The KOA list of files has 335474 files, which have 1784 unique observation dates.


In [124]:
# Determine the average epoch time
ave_time = (megafile['epoch_ra_gaia'] + megafile['epoch_dec_gaia']) / 2

In [125]:
# Set up catalog of all the stars in the megafile
c = SkyCoord(ra = list(megafile['ra'])*u.deg, dec = list(megafile['dec'])*u.deg, 
distance = Distance(parallax = list(megafile['gaia_parallax']) * u.mas), 
pm_ra_cosdec = list(megafile['pmra_gaia']) * u.mas/u.yr, 
pm_dec = list(megafile['pmdec_gaia']) * u.mas/u.yr, 
obstime = Time(ave_time, format='decimalyear'))
print(len(c))

115663


In [126]:
# Breaking down the KOA file into groups by date
a = 0
for i in myset:
    df = koafile[koafile["date"] == i]
    df.to_csv('/Users/Jess/HGCA_survey_paper/koa_date' + str(a) + '.csv')
    a = a + 1

In [127]:
# Setting timer for segment
t0 = time.time()

# list to save final indexes
total_indices = []

# Opening each of the files - there is one for each date, each containing all observations at that date. 
koa_dates = glob.glob('/Users/Jess/HGCA_survey_paper/koa_date*.csv')

# Loop through each date 
for i in koa_dates:
    koad = pd.read_csv(i) # Date file generally 
    print('There are ' + str(len(koad)) + ' KOA observations with this date.')
    
    koadd = pd.read_csv(i)['date'][0] # The specific date in this file
    print('Shifting HGCA stars to date No#: ' + str(koadd)) 
    
    # Set up catalog of all the stars in the megafile
    c = SkyCoord(ra = list(megafile['ra'])*u.deg, dec = list(megafile['dec'])*u.deg, 
    distance = Distance(parallax = list(megafile['gaia_parallax']) * u.mas), 
    pm_ra_cosdec = list(megafile['pmra_gaia']) * u.mas/u.yr, 
    pm_dec = list(megafile['pmdec_gaia']) * u.mas/u.yr, 
    obstime = Time(ave_time, format='decimalyear'))
   
    # Shift the megafile to the new observation date
    sc = c.apply_space_motion(new_obstime=Time(koadd))
        
    # Create a catalog for the KOA observations at this date, removing the 'NaN' values
    catalog = SkyCoord(ra=koad['ra']*u.degree, dec=koad['dec']*u.degree)
    clean = (~np.isnan(catalog.ra) & ~np.isnan(catalog.dec))
    catalog_clean = catalog[clean]
        
    # Search around the sky 
    idxcatalog, idxsc, d2d, d3d = sc.search_around_sky(catalog_clean, 20*u.arcsecond)
    print('There are ' + str(len(set(idxsc))) + ' stars in HGCA that are also present in KOA observations. \n')
    
    # Save indices for matches found
    for x in idxsc:
        total_indices.append(x)

# Complete timer
t1 = time.time()
total = t1-t0
print('The time to run this code segment is ' + str(total/60) + 'minutes.')

There are 146 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-04-19
There are 6 stars in HGCA that are also present in KOA observations. 

There are 384 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-06-17
There are 34 stars in HGCA that are also present in KOA observations. 

There are 521 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-06-26
There are 2 stars in HGCA that are also present in KOA observations. 

There are 32 KOA observations with this date.
Shifting HGCA stars to date No#: 2006-06-29
There are 0 stars in HGCA that are also present in KOA observations. 

There are 846 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-07-27
There are 19 stars in HGCA that are also present in KOA observations. 

There are 210 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-05-19
There are 0 stars in HGCA that are also present in KOA observations. 

There are 153 KOA observati

There are 1 stars in HGCA that are also present in KOA observations. 

There are 6 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-02-05
There are 1 stars in HGCA that are also present in KOA observations. 

There are 70 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-07-23
There are 2 stars in HGCA that are also present in KOA observations. 

There are 15 KOA observations with this date.
Shifting HGCA stars to date No#: 2003-07-26
There are 2 stars in HGCA that are also present in KOA observations. 

There are 125 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-08-06
There are 2 stars in HGCA that are also present in KOA observations. 

There are 12 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-01-25
There are 1 stars in HGCA that are also present in KOA observations. 

There are 13 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-06-03
There are 0 stars in HGCA that are

There are 1 stars in HGCA that are also present in KOA observations. 

There are 212 KOA observations with this date.
Shifting HGCA stars to date No#: 2004-09-09
There are 1 stars in HGCA that are also present in KOA observations. 

There are 228 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-03-18
There are 0 stars in HGCA that are also present in KOA observations. 

There are 34 KOA observations with this date.
Shifting HGCA stars to date No#: 2001-07-31
There are 0 stars in HGCA that are also present in KOA observations. 

There are 280 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-06-18
There are 0 stars in HGCA that are also present in KOA observations. 

There are 303 KOA observations with this date.
Shifting HGCA stars to date No#: 2015-11-18
There are 7 stars in HGCA that are also present in KOA observations. 

There are 16 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-05-30
There are 0 stars in HGCA that

There are 0 stars in HGCA that are also present in KOA observations. 

There are 109 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-10-06
There are 6 stars in HGCA that are also present in KOA observations. 

There are 96 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-10-29
There are 0 stars in HGCA that are also present in KOA observations. 

There are 482 KOA observations with this date.
Shifting HGCA stars to date No#: 2013-07-18
There are 0 stars in HGCA that are also present in KOA observations. 

There are 486 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-06-21
There are 2 stars in HGCA that are also present in KOA observations. 

There are 99 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-11-25
There are 0 stars in HGCA that are also present in KOA observations. 

There are 166 KOA observations with this date.
Shifting HGCA stars to date No#: 2004-07-27
There are 1 stars in HGCA that

There are 13 stars in HGCA that are also present in KOA observations. 

There are 38 KOA observations with this date.
Shifting HGCA stars to date No#: 2006-10-15
There are 1 stars in HGCA that are also present in KOA observations. 

There are 35 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-01-18
There are 3 stars in HGCA that are also present in KOA observations. 

There are 54 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-05-24
There are 0 stars in HGCA that are also present in KOA observations. 

There are 118 KOA observations with this date.
Shifting HGCA stars to date No#: 2006-11-16
There are 0 stars in HGCA that are also present in KOA observations. 

There are 2 KOA observations with this date.
Shifting HGCA stars to date No#: 2018-02-22
There are 0 stars in HGCA that are also present in KOA observations. 

There are 51 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-10-05
There are 0 stars in HGCA that ar

...

There are 0 stars in HGCA that are also present in KOA observations. 

There are 20 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-07-24
There are 1 stars in HGCA that are also present in KOA observations. 

There are 135 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-08-15
There are 1 stars in HGCA that are also present in KOA observations. 

There are 10 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-04-28
There are 0 stars in HGCA that are also present in KOA observations. 

There are 155 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-05-18
There are 1 stars in HGCA that are also present in KOA observations. 

There are 42 KOA observations with this date.
Shifting HGCA stars to date No#: 2015-03-30
There are 1 stars in HGCA that are also present in KOA observations. 

There are 261 KOA observations with this date.
Shifting HGCA stars to date No#: 2008-09-08
There are 1 stars in HGCA that 

There are 1 stars in HGCA that are also present in KOA observations. 

There are 8 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-03-29
There are 0 stars in HGCA that are also present in KOA observations. 

There are 26 KOA observations with this date.
Shifting HGCA stars to date No#: 2002-03-05
There are 0 stars in HGCA that are also present in KOA observations. 

There are 20 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-12-23
There are 1 stars in HGCA that are also present in KOA observations. 

There are 156 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-05-09
There are 0 stars in HGCA that are also present in KOA observations. 

There are 568 KOA observations with this date.
Shifting HGCA stars to date No#: 2002-09-23
There are 3 stars in HGCA that are also present in KOA observations. 

There are 27 KOA observations with this date.
Shifting HGCA stars to date No#: 2012-06-07
There are 0 stars in HGCA that ar

There are 6 stars in HGCA that are also present in KOA observations. 

There are 173 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-09-07
There are 1 stars in HGCA that are also present in KOA observations. 

There are 45 KOA observations with this date.
Shifting HGCA stars to date No#: 2003-08-19
There are 2 stars in HGCA that are also present in KOA observations. 

There are 455 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-05-05
There are 0 stars in HGCA that are also present in KOA observations. 

There are 69 KOA observations with this date.
Shifting HGCA stars to date No#: 2004-05-24
There are 0 stars in HGCA that are also present in KOA observations. 

There are 323 KOA observations with this date.
Shifting HGCA stars to date No#: 2016-06-16
There are 0 stars in HGCA that are also present in KOA observations. 

There are 251 KOA observations with this date.
Shifting HGCA stars to date No#: 2001-12-18
There are 11 stars in HGCA tha

There are 12 stars in HGCA that are also present in KOA observations. 

There are 194 KOA observations with this date.
Shifting HGCA stars to date No#: 2003-12-09
There are 12 stars in HGCA that are also present in KOA observations. 

There are 283 KOA observations with this date.
Shifting HGCA stars to date No#: 2017-05-08
There are 0 stars in HGCA that are also present in KOA observations. 

There are 34 KOA observations with this date.
Shifting HGCA stars to date No#: 2007-12-15
There are 2 stars in HGCA that are also present in KOA observations. 

There are 400 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-06-01
There are 12 stars in HGCA that are also present in KOA observations. 

There are 264 KOA observations with this date.
Shifting HGCA stars to date No#: 2009-06-20
There are 1 stars in HGCA that are also present in KOA observations. 

There are 20 KOA observations with this date.
Shifting HGCA stars to date No#: 2008-05-14
There are 0 stars in HGCA t

There are 1 stars in HGCA that are also present in KOA observations. 

There are 140 KOA observations with this date.
Shifting HGCA stars to date No#: 2010-05-01
There are 0 stars in HGCA that are also present in KOA observations. 

There are 181 KOA observations with this date.
Shifting HGCA stars to date No#: 2004-04-04
There are 10 stars in HGCA that are also present in KOA observations. 

There are 122 KOA observations with this date.
Shifting HGCA stars to date No#: 2015-05-09
There are 0 stars in HGCA that are also present in KOA observations. 

There are 109 KOA observations with this date.
Shifting HGCA stars to date No#: 2011-07-09
There are 2 stars in HGCA that are also present in KOA observations. 

There are 174 KOA observations with this date.
Shifting HGCA stars to date No#: 2005-11-25
There are 0 stars in HGCA that are also present in KOA observations. 

There are 382 KOA observations with this date.
Shifting HGCA stars to date No#: 2014-12-06
There are 5 stars in HGCA t

In [113]:
# Matching the indices to rows in the megafile and saving as a new csv.
print('There are ' + str(len(set(total_indices))) + ' unique stars in this list.')
idx_set = list(set(total_indexes))
cross_matched_stars = megafile.iloc[idx_set]
cross_matched_stars.to_csv('/Users/Jess/HGCA_survey_paper/completed_cross_match.csv')